In [24]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [25]:
from sklearn.pipeline import make_pipeline

In [26]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='s3://mlflow-models-david/1', experiment_id='1', lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [27]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [28]:
df_train = read_dataframe('./data/green_tripdata_2022-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2022-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [29]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.101201727016495


In [30]:
from mlflow.tracking import MlflowClient


In [33]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '95b544bd26ca48f1b417cfc304ee9114'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [34]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

MlflowException: The following failures occurred while downloading one or more artifacts from s3://mlflow-models-david/1/95b544bd26ca48f1b417cfc304ee9114/artifacts: {'dict_vectorizer.bin': "ClientError('An error occurred (404) when calling the HeadObject operation: Not Found')"}

In [ ]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [ ]:
dv

DictVectorizer()